# Training simple model and evalualing its predictions on different tasks

## Prepare dataset for training

First let's load splitted dataset generated in [another notebook](https://github.com/tinyclues/recsys-multi-atrribute-benchmark/blob/master/dataset_preprocessing/movielens%20with%20imdb.ipynb)

In [1]:
DATASET = 'movielens_imdb'

In [2]:
from utils import load_dataset

datasets = {}
for split_name in ['train', 'val', 'test']:
    datasets[split_name] = load_dataset(DATASET, split_name)

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


We can parse features' names, they were chosen to easily distinguish between offer features (that will be used to modelize film) and user features (aggregated history up to chosen date).

In [3]:
from utils import AGG_PREFIX

all_columns = list(datasets['train'].element_spec.keys())
technical_columns = ['userId', 'date']
user_features = list(filter(lambda x: x.startswith(AGG_PREFIX), all_columns))
offer_features = list(filter(lambda x: x not in user_features + technical_columns, all_columns))

In [4]:
user_features

['aggregated_ratings_imdbId',
 'aggregated_ratings_titleType',
 'aggregated_ratings_genre',
 'aggregated_ratings_runtimeMinutesCluster',
 'aggregated_ratings_director',
 'aggregated_ratings_actor',
 'aggregated_ratings_startYearCluster']

In [5]:
offer_features

['genre',
 'actor',
 'director',
 'imdbId',
 'titleType',
 'runtimeMinutesCluster',
 'startYearCluster']

### Rebatch dataset by events

First we will unnest events for each user (stored in second dimension of saved tensors) and keep only limited number of them. This operation will be needed further to avoid collisions during generation of negative examples. Then we will rebatch results into smaller batches (`50400` events for validation and test sets and `10080` events for train set).

In [6]:
%%time

from functools import partial
from uuid import uuid4

from utils import rebatch_by_events

datasets['train'] = rebatch_by_events(datasets['train'], batch_size=10080, date_column='date', nb_events_by_user_by_day=8)
for key in ['val', 'test']:
    datasets[key] = rebatch_by_events(datasets[key], batch_size=50400, date_column='date', nb_events_by_user_by_day=8,
                                      seed=1729).cache(f'/tmp/{uuid4()}.tf')

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
CPU times: user 40.6 s, sys: 6.18 s, total: 46.7 s
Wall time: 34.2 s


In [7]:
train_batch, y = next(iter(datasets['train']))
train_batch['imdbId'].shape[0]  # check batch size

10080

## Define simple model

Let's now define a simple model we want to test. Independetly from model's choice we need to embed inputs in some vectorial space. To define such embeddings we need number of different modalities inputs can take, and we can get this information from saved vectorizers:

In [8]:
from utils import load_inverse_lookups
inverse_lookups = load_inverse_lookups(DATASET)

In [9]:
import re

vocabulary_sizes = {}

for feature in offer_features:
    vocabulary_sizes[feature] = inverse_lookups[feature].vocabulary_size()

for feature in user_features:
    for key in inverse_lookups:
        pattern = re.compile(r"{}(\w+)_{}".format(AGG_PREFIX, key))
        if pattern.match(feature):
            vocabulary_sizes[feature] = vocabulary_sizes[key]

Now `vocabulary_sizes` contains modality of each feature

In [10]:
vocabulary_sizes

{'genre': 36,
 'actor': 2510,
 'director': 3095,
 'imdbId': 7894,
 'titleType': 20,
 'runtimeMinutesCluster': 35,
 'startYearCluster': 40,
 'aggregated_ratings_imdbId': 7894,
 'aggregated_ratings_titleType': 20,
 'aggregated_ratings_genre': 36,
 'aggregated_ratings_runtimeMinutesCluster': 35,
 'aggregated_ratings_director': 3095,
 'aggregated_ratings_actor': 2510,
 'aggregated_ratings_startYearCluster': 40}

### Model architecture

In [11]:
import tensorflow as tf

For the benchmarks we want to do, model's architecture doesn't play a crucial role, we saw the same problems in any model that averages embeddings of offer features in a naive way. So let's take some simple model's architecture, for example collaborative filtering using two towers neural network:

<img src="resources/two_towers_model.png" alt="two tower model" width="800" />

### Model parameters

To choose model's parameters we did some manual tuning using validation set to maximize train and validation AUC while keeping mismatch between them small.

In [12]:
import tensorflow as tf

# model parameters
EMBEDDING_DIM = 100
L1_COEFF = 8.5e-7
DROPOUT = 0.17


def REGULARIZER():
    return {'class_name': 'L1L2', 'config': {'l1': L1_COEFF, 'l2': 0.}}

def USER_TOWER():
    return tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(80,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(40,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
    ], name='user_tower')

def OFFER_TOWER():
    return tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(80,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
        tf.keras.layers.Dense(40,
                              kernel_regularizer=REGULARIZER(),
                              bias_regularizer=REGULARIZER()),
        tf.keras.layers.Dropout(DROPOUT),
        tf.keras.layers.Activation('tanh'),
    ], name='offer_tower')

EPOCHS = 12

NUMBER_OF_NEGATIVES = 4
LOSS = tf.keras.losses.BinaryCrossentropy(from_logits=True)
AUC_METRIC = tf.keras.metrics.AUC(from_logits=True)

import tensorflow_addons as tfa
OPTIMIZER = tfa.optimizers.AdamW(weight_decay=8.5e-8, learning_rate=0.0008)

### Embeddings

Let's embed all available `user_features` and `offer_features` into vectorial space of dimension `EMBEDDING_DIM`. We use custom embeddings layer class `WeightedEmbeddings` that will automatically take a mean embedding vector when needed.

In particular,
* `user_features` are lists of attributes and we don't need to take into account any weights.
* `offer_features` during the inference can contain lists of attributes because of aggregation. We will also pass weights explicitly during the inference
* `offer_features` during training are lists with only one element, so we need to define dummy weights for training

All three cases can be treated by the same layer, where we will define a sparse matrix of all attributes we want to embed and then multiply it by the dense matrix with embeddings, multiplying by weights at the same time (if needed).

In [13]:
from utils import add_equal_weights

for key in datasets:
    datasets[key] = datasets[key].map(partial(add_equal_weights, features=offer_features))
train_batch, y = next(iter(datasets['train']))

In [14]:
# dummy weights needed for training
train_batch['genre_weight'][:5]

<tf.RaggedTensor [[1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0]]>

In [15]:
# embeddings layer example
from layers import WeightedEmbeddings
example_layer = WeightedEmbeddings(3, 5, name='test')
example_layer(tf.ragged.constant([[0], [1], [0, 1]]))

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[ 0.04494161, -0.01325916, -0.02227489,  0.03725203, -0.04359887],
       [ 0.03168532,  0.00576754,  0.00114274,  0.02043159, -0.04843969],
       [ 0.03831346, -0.00374581, -0.01056607,  0.02884181, -0.04601928]],
      dtype=float32)>

Now we can define all embeddings layers:

In [16]:
from layers import get_input_layer, WeightedEmbeddings
from utils import WEIGHT_SUFFIX

embeddings, inputs = {}, {}
for feature in user_features + offer_features:
    if feature in offer_features:
        # for offer features we need weights:
        # with dummy weights during training, and the ones used for a feature's averaging at inference time
        inputs[f'{feature}{WEIGHT_SUFFIX}'] = get_input_layer(f'{feature}{WEIGHT_SUFFIX}', tf.float32)
    inputs[feature] = get_input_layer(feature)
    # here we use input feature modality from `vocabulary_sizes` to know embeddings matrix dimensions
    emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                   EMBEDDING_DIM, name=f'{feature}_embedding',
                                   embeddings_regularizer=REGULARIZER())
    embeddings[feature] = emb_layer(inputs[feature], inputs.get(f'{feature}{WEIGHT_SUFFIX}'))

In [17]:
embeddings

{'aggregated_ratings_imdbId': <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'aggregated_ratings_imdbId_embedding')>,
 'aggregated_ratings_titleType': <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'aggregated_ratings_titleType_embedding')>,
 'aggregated_ratings_genre': <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'aggregated_ratings_genre_embedding')>,
 'aggregated_ratings_runtimeMinutesCluster': <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'aggregated_ratings_runtimeMinutesCluster_embedding')>,
 'aggregated_ratings_director': <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'aggregated_ratings_director_embedding')>,
 'aggregated_ratings_actor': <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'aggregated_ratings_actor_embedding')>,
 'aggregated_ratings_startYearCluster': <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'aggregated_ratings_startYearCluster_embedding')>,

### Combining everything into model

Now we can define described model architecture on the top of embeddings.

In [18]:
embedded_user_features = [embeddings[feature] for feature in user_features]
embedded_offer_features = [embeddings[feature] for feature in offer_features]
user_tower = USER_TOWER()(tf.keras.layers.Concatenate(name='concat_user')(embedded_user_features))
offer_tower = OFFER_TOWER()(tf.keras.layers.Concatenate(name='concat_offer')(embedded_offer_features))

### Negative generation in mini-batches

As our dataset contains only positive examples, up to this point we used only them. We have different choices of how to choose negative examples, but we chose most optimal one for calculations (both in memory and time) - we will generate negatives at the same time as calculating interactions between user and offer embeddings, proceding in minibatches:
* let's fix a number `N - 1` of how many negative examples we want to generate for each positive one
* consider minibatches of size `N` with events done on the same (or close) date (this was ensured by batch construction above), so we will get negative example from the actions on the same date as the positive one, similar to [Contrastive Predictive Coding](https://arxiv.org/abs/1807.03748) method.
* inside each minibatch we have users `u1, u2, ..., uN` who rated films `f1, f2, ..., fN` respectively on the same date `d`
* let's consider all possible pairs `(u1, f1), (u1, f2), ..., (uN, fN)` (`N ** 2` pairs in total)
* among those pairs there are `N` positive examples, all other `N(N - 1)` pairs are considered as negative ones
* it gives us exactly `N - 1` negative examples for each from `N` positive ones

We pair this process with interaction calculation by calculating not only scalar products between positive pairs, but between all `N ** 2` pairs per minibatch. Such operation can be written as multiplication of tensors, keeping number of embeddings calculations fixed (`2 * N` for each minibatch).

In [19]:
class DotWithNegatives(tf.keras.layers.Layer):
    def __init__(self, number_of_negatives, **kwargs):
        super().__init__(**kwargs)
        self.number_of_negatives = number_of_negatives
        
    def call(self, inputs, generate_negatives):
        user_embeddings, offer_embeddings = inputs
        if generate_negatives:
            # here we will generate negative examples inside mini-batches
            batch_size = tf.shape(user_embeddings)[0]
            # we split original batch into mini-batches of size (number_of_negatives + 1)
            minibatch_shape = (batch_size // (self.number_of_negatives + 1), (self.number_of_negatives + 1), -1)
            user_embeddings = tf.reshape(user_embeddings, minibatch_shape)
            offer_embeddings = tf.reshape(offer_embeddings, minibatch_shape)
            # for each pair of lines i,j inside minibatch, we consider pairs user/offer
            # * as positive examples when i==j
            # * as negative examples otherwise
            # at the end we flatten mini-batch dimension and obtain batch_size * (number_of_negatives + 1) predictions
            res = tf.einsum('bid,bjd->bij', user_embeddings, offer_embeddings)
        else:
            # otherwise we do just scalar product, let's write it in einsum notation too to see a difference between two
            res = tf.einsum('bd,bd->b', user_embeddings, offer_embeddings)
        return tf.reshape(res, (-1, 1))

In [20]:
# we don't apply sigmoid on the output and will have from_logits=True in both loss and metrics
output = DotWithNegatives(NUMBER_OF_NEGATIVES, name='prediction')([user_tower, offer_tower], generate_negatives=True)

In [21]:
output

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'prediction')>

Now our labels from batch are not aligned with output we produce, to get positive/negative labels at needed positions we just need to find which index would correspond to which label when considering a minibatch. This logic is implemented in auxilary classes `BroadcastLoss` and `BroadcastMetric`.

In [22]:
from utils import BroadcastLoss, BroadcastMetric

model = tf.keras.Model(inputs, output, name='two_tower_model')
model.compile(optimizer=OPTIMIZER,
              loss=BroadcastLoss(LOSS, NUMBER_OF_NEGATIVES),
              metrics=[BroadcastMetric(AUC_METRIC, NUMBER_OF_NEGATIVES)])

In [23]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file=f'models/{DATASET}_simple_model.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


### Training

In [24]:
model.fit(datasets['train'], epochs=EPOCHS, validation_data=datasets['val'])

Epoch 1/12


/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['date', 'userId'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


231/231 [==============================] - 141s 396ms/step - loss: 0.5597 - auc: 0.5648 - val_loss: 0.5736 - val_auc: 0.5315
Epoch 2/12
231/231 [==============================] - 87s 343ms/step - loss: 0.5079 - auc: 0.6202 - val_loss: 0.5386 - val_auc: 0.6108
Epoch 3/12
231/231 [==============================] - 91s 359ms/step - loss: 0.4993 - auc: 0.6281 - val_loss: 0.5077 - val_auc: 0.6342
Epoch 4/12
231/231 [==============================] - 91s 366ms/step - loss: 0.4957 - auc: 0.6318 - val_loss: 0.4966 - val_auc: 0.6373
Epoch 5/12
231/231 [==============================] - 88s 349ms/step - loss: 0.4934 - auc: 0.6341 - val_loss: 0.4945 - val_auc: 0.6379
Epoch 6/12
231/231 [==============================] - 88s 345ms/step - loss: 0.4915 - auc: 0.6363 - val_loss: 0.4929 - val_auc: 0.6393
Epoch 7/12
231/231 [==============================] - 84s 331ms/step - loss: 0.4900 - auc: 0.6381 - val_loss: 0.4927 - val_auc: 0.6387
Epoch 8/12
231/231 [==============================] - 87s 340ms/s

## Single task models benchmark

As described in (TODO link to article) we can consider predictions on one chosen offer column as a single task and the whole setup as a multi-task problem. Let's now evaluate performance of a common model on a subset of tasks. We will compare its results against single task models sharing the same architecture, but using only one offer feature at time.

In [25]:
# offer columns we want to evaluate, specific to dataset we test
TASKS = ['imdbId', 'director', 'genre']

For simplicity of further code, let's wrap whole model definition into a function:

In [26]:
def two_tower_model(offer_features, name='two_tower_model'):
    # user_features, vocabulary_sizes, EMBEDDING_DIM, REGULARIZER, USER_TOWER, OFFER_TOWER,
    # OPTIMIZER, LOSS, NUMBER_OF_NEGATIVES
    # come from global scope, but can be passed as params instead
    embeddings, inputs = {}, {}
    for feature in user_features + offer_features:
        if feature in offer_features:
            # for offer features we need weights:
            # with dummy weights during training, and the ones used for a feature's averaging at inference time
            inputs[f'{feature}{WEIGHT_SUFFIX}'] = get_input_layer(f'{feature}{WEIGHT_SUFFIX}', tf.float32)
        inputs[feature] = get_input_layer(feature)
        # here we use input feature modality from `vocabulary_sizes` to know embeddings matrix dimensions
        emb_layer = WeightedEmbeddings(vocabulary_sizes[feature],
                                       EMBEDDING_DIM, name=f'{feature}_embedding',
                                       embeddings_regularizer=REGULARIZER())
        embeddings[feature] = emb_layer(inputs[feature], inputs.get(f'{feature}{WEIGHT_SUFFIX}'))
    
    embedded_user_features = [embeddings[feature] for feature in user_features]
    embedded_offer_features = [embeddings[feature] for feature in offer_features]
    user_tower = USER_TOWER()(tf.keras.layers.Concatenate(name='concat_user')(embedded_user_features))
    offer_tower = OFFER_TOWER()(tf.keras.layers.Concatenate(name='concat_offer')(embedded_offer_features))
    
    output = DotWithNegatives(NUMBER_OF_NEGATIVES, name='prediction')([user_tower, offer_tower], generate_negatives=True)
    model = tf.keras.Model(inputs, output, name=name)
    model.compile(optimizer=OPTIMIZER,
                  loss=BroadcastLoss(LOSS, NUMBER_OF_NEGATIVES),
                  metrics=[BroadcastMetric(AUC_METRIC, NUMBER_OF_NEGATIVES)])
    
    return model

We train models that use only one offer feature with same hyperparameters as the initial model.

In [27]:
mono_feature_models = {}
for task_offer_feature in TASKS:
    mono_feature_models[task_offer_feature] = two_tower_model([task_offer_feature],
                                                              name=f'{task_offer_feature}_model')
    mono_feature_models[task_offer_feature].fit(datasets['train'],
                                                epochs=EPOCHS,
                                                validation_data=datasets['val'])

Epoch 1/12


/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['genre', 'actor', 'date', 'director', 'userId', 'titleType', 'runtimeMinutesCluster', 'startYearCluster', 'genre_weight', 'actor_weight', 'director_weight', 'titleType_weight', 'runtimeMinutesCluster_weight', 'startYearCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


231/231 [==============================] - 98s 305ms/step - loss: 0.5340 - auc: 0.5744 - val_loss: 0.5319 - val_auc: 0.5305
Epoch 2/12
231/231 [==============================] - 69s 268ms/step - loss: 0.5014 - auc: 0.6170 - val_loss: 0.5224 - val_auc: 0.6040
Epoch 3/12
231/231 [==============================] - 69s 269ms/step - loss: 0.4965 - auc: 0.6239 - val_loss: 0.5015 - val_auc: 0.6262
Epoch 4/12
231/231 [==============================] - 70s 274ms/step - loss: 0.4934 - auc: 0.6293 - val_loss: 0.4932 - val_auc: 0.6319
Epoch 5/12
231/231 [==============================] - 70s 277ms/step - loss: 0.4912 - auc: 0.6333 - val_loss: 0.4909 - val_auc: 0.6339
Epoch 6/12
231/231 [==============================] - 70s 276ms/step - loss: 0.4895 - auc: 0.6358 - val_loss: 0.4900 - val_auc: 0.6351
Epoch 7/12
231/231 [==============================] - 70s 274ms/step - loss: 0.4880 - auc: 0.6379 - val_loss: 0.4884 - val_auc: 0.6368
Epoch 8/12
231/231 [==============================] - 71s 276ms/st

/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['genre', 'actor', 'date', 'imdbId', 'userId', 'titleType', 'runtimeMinutesCluster', 'startYearCluster', 'genre_weight', 'actor_weight', 'imdbId_weight', 'titleType_weight', 'runtimeMinutesCluster_weight', 'startYearCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


231/231 [==============================] - 96s 320ms/step - loss: 0.5266 - auc: 0.5685 - val_loss: 0.5231 - val_auc: 0.5166
Epoch 2/12
231/231 [==============================] - 72s 283ms/step - loss: 0.5006 - auc: 0.5987 - val_loss: 0.5149 - val_auc: 0.5763
Epoch 3/12
231/231 [==============================] - 71s 284ms/step - loss: 0.4974 - auc: 0.6034 - val_loss: 0.5015 - val_auc: 0.6094
Epoch 4/12
231/231 [==============================] - 71s 277ms/step - loss: 0.4956 - auc: 0.6062 - val_loss: 0.4955 - val_auc: 0.6119
Epoch 5/12
231/231 [==============================] - 71s 276ms/step - loss: 0.4942 - auc: 0.6084 - val_loss: 0.4948 - val_auc: 0.6123
Epoch 6/12
231/231 [==============================] - 71s 275ms/step - loss: 0.4931 - auc: 0.6099 - val_loss: 0.4933 - val_auc: 0.6139
Epoch 7/12
231/231 [==============================] - 70s 275ms/step - loss: 0.4920 - auc: 0.6118 - val_loss: 0.4924 - val_auc: 0.6147
Epoch 8/12
231/231 [==============================] - 71s 279ms/st

/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['actor', 'date', 'director', 'imdbId', 'userId', 'titleType', 'runtimeMinutesCluster', 'startYearCluster', 'actor_weight', 'director_weight', 'imdbId_weight', 'titleType_weight', 'runtimeMinutesCluster_weight', 'startYearCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


231/231 [==============================] - 101s 323ms/step - loss: 0.5341 - auc: 0.5259 - val_loss: 0.5078 - val_auc: 0.5041
Epoch 2/12
231/231 [==============================] - 71s 278ms/step - loss: 0.5083 - auc: 0.5289 - val_loss: 0.5054 - val_auc: 0.5200
Epoch 3/12
231/231 [==============================] - 71s 276ms/step - loss: 0.5062 - auc: 0.5313 - val_loss: 0.5038 - val_auc: 0.5435
Epoch 4/12
231/231 [==============================] - 71s 278ms/step - loss: 0.5053 - auc: 0.5321 - val_loss: 0.5032 - val_auc: 0.5444
Epoch 5/12
231/231 [==============================] - 74s 289ms/step - loss: 0.5046 - auc: 0.5333 - val_loss: 0.5032 - val_auc: 0.5450
Epoch 6/12
231/231 [==============================] - 71s 275ms/step - loss: 0.5041 - auc: 0.5337 - val_loss: 0.5027 - val_auc: 0.5456
Epoch 7/12
231/231 [==============================] - 70s 274ms/step - loss: 0.5037 - auc: 0.5346 - val_loss: 0.5022 - val_auc: 0.5456
Epoch 8/12
231/231 [==============================] - 73s 286ms/s

## Evaluation

Now let's generate some offers from test dataset:
* we will consider all batches from test dataset
* we perform a group by using each feature from `TASKS` as a group by key
* for all offer features except the one we are using as key we generate ragged tensors with bag of values it can take
* we remove least popular values in each list
* so now each line of dataset corresponds to an offer of type `task_offer_feature = 'value'`

In [28]:
%%time
from utils import prepare_single_task_dataset
test_datasets = {}
for task_offer_feature in TASKS:
    test_datasets[task_offer_feature] = \
        prepare_single_task_dataset(datasets['test'], task_offer_feature, offer_features)

CPU times: user 1min 55s, sys: 8.03 s, total: 2min 3s
Wall time: 1min 52s


Test dataset for a given task keeps a column used for group by as is, but other offer columns become lists (to encode bag of values) and we need to average embeddings for them:

In [29]:
test_batch, y = next(iter(test_datasets['genre']))

In [30]:
test_batch['genre'][:5]

<tf.RaggedTensor [[10],
 [10],
 [14],
 [14],
 [11]]>

In [31]:
test_batch['genre_weight'][:5]

<tf.RaggedTensor [[1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0]]>

In [32]:
test_batch['director'][:1]

<tf.RaggedTensor [[312, 0, 546, 790, 903, 835, 513, 786, 537, 321, 298, 842, 7, 773, 548,
  318, 655, 310, 924, 155, 562, 659, 2, 387, 455, 217, 613, 615, 512, 232,
  889, 225, 483, 358, 436, 274, 471, 493, 698, 438, 177, 108, 473, 488,
  491, 111, 6, 269, 42, 253, 348, 774, 534, 5, 9, 77, 8, 575, 245, 516,
  399, 349, 341, 775, 442, 467, 541, 472, 84, 634, 416, 139, 418, 377,
  452, 18, 660, 439, 409, 417, 432, 601, 644, 226, 408, 664, 268, 291,
  480, 376, 243, 476, 23, 517, 113, 629, 170, 670, 197, 362, 48, 230, 271,
  124, 440, 121, 214, 328, 167, 391, 149, 371, 375, 344, 252, 627, 313,
  431, 286, 583, 169, 98, 277, 174, 370, 258, 293, 165, 482, 330, 294,
  244, 285, 218, 396, 289, 228, 381, 237, 554, 107, 411, 148, 306, 205,
  130, 47, 203, 194, 337, 254, 105, 208, 216, 128, 182, 92, 403, 199, 248,
  150, 275, 173, 406, 247, 260, 372, 212, 320, 16, 93, 355, 202, 240, 272,
  120, 331, 82, 131, 44, 129, 55, 184, 196, 96, 156, 235, 397, 266, 24,
  134, 211, 163, 160, 181, 32, 249, 1

In [33]:
test_batch['director_weight'][:1]

<tf.RaggedTensor [[0.00089855335, 0.00089855335, 0.00089855335, 0.00089855335,
  0.00089855335, 0.00089855335, 0.00092101714, 0.00092101714, 0.000943481,
  0.000943481, 0.000943481, 0.000943481, 0.00096594484, 0.00096594484,
  0.0009884087, 0.0009884087, 0.0010108725, 0.0010108725, 0.0010108725,
  0.0010108725, 0.0010333363, 0.0010333363, 0.0010558001, 0.0010558001,
  0.0010558001, 0.0010558001, 0.0010558001, 0.0010558001, 0.001078264,
  0.001078264, 0.001078264, 0.0011007278, 0.0011007278, 0.0011007278,
  0.0011231917, 0.0011231917, 0.0011231917, 0.0011231917, 0.0011231917,
  0.0011456555, 0.0011456555, 0.0011456555, 0.0011456555, 0.0011456555,
  0.0011456555, 0.0011681194, 0.0011905831, 0.0011905831, 0.0011905831,
  0.001213047, 0.001213047, 0.0012355108, 0.0012579747, 0.0012579747,
  0.0012804385, 0.0013029024, 0.0013029024, 0.0013029024, 0.0013253662,
  0.0013253662, 0.0013253662, 0.0013253662, 0.0013253662, 0.0013253662,
  0.0013478299, 0.0013702938, 0.0013702938, 0.0013702938, 0.

Now we can apply model on grouped features for each task and calculate AUC for each offer of type `task_offer_feature = 'value'`. Note, that negatives are generated in the same way as for training.

In [34]:
%%time
from collections import defaultdict
from utils import evaluate_model, wAUC

aucs = defaultdict(dict)
for task_offer_feature in TASKS:
    for model_name in TASKS:
        aucs[task_offer_feature][f'MONO:{model_name}'] = \
            evaluate_model(mono_feature_models[model_name],
                           task_offer_feature, test_datasets, NUMBER_OF_NEGATIVES, inverse_lookups)
    aucs[task_offer_feature]['simple model'] = \
            evaluate_model(model, task_offer_feature, test_datasets, NUMBER_OF_NEGATIVES, inverse_lookups)

/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['genre', 'actor', 'date', 'imdbId', 'userId', 'titleType', 'runtimeMinutesCluster', 'startYearCluster', 'genre_weight', 'actor_weight', 'imdbId_weight', 'titleType_weight', 'runtimeMinutesCluster_weight', 'startYearCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.9/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['actor', 'date', 'director', 'imdbId', 'userId', 'titleType', 'runtimeMinutesCluster', 'startYearCluster', 'actor_weight', 'director_weight', 'imdbId_weight', 'titleType_weight', 'runtimeMinutesCluster_weight', 'startYearCluster_weight'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.9/site-packages/keras/engine/functi

CPU times: user 12min 32s, sys: 1min 28s, total: 14min 1s
Wall time: 3min 45s


In [35]:
aucs['imdbId']['MONO:imdbId'][10:].head()

,auc,name,number of events
group_idx,,,
10,0.661372,The Shawshank Redemption,1183
11,0.624429,Pulp Fiction,1052
12,0.556605,The Silence of the Lambs,979
13,0.573551,Forrest Gump,928
14,0.602212,Star Wars: Episode IV - A New Hope,787


We can aggregate AUCs from individual offers to have one value we can compare among models: weighted macro AUC. We will keep only offers with more than 200 positive events and weight their AUCs by number of events:

In [46]:
import pandas as pd
results = pd.DataFrame()
for task_name in aucs:
    for model_name in aucs[task_name]:
        w_auc = wAUC(aucs[task_name][model_name])
        results = pd.concat([results,
                             pd.Series({'wAUC': w_auc, 'offers': task_name, 'model': model_name}).to_frame().T],
                            ignore_index=True)

In [47]:
pd.pivot_table(results, 'wAUC', 'model', 'offers').style.background_gradient(cmap='coolwarm')

offers,director,genre,imdbId
model,,,
MONO:director,0.586179,0.539052,0.587248
MONO:genre,0.538615,0.551291,0.512951
MONO:imdbId,0.586306,0.531941,0.599211
simple model,0.586910,0.534218,0.603315
